# Load libraries

In [35]:
library("jsonlite")
library("knitr")
library("dataRetrieval")
# Adjust dsn to local directory
dsn = "C:\\Users\\jmatney\\Documents\\R2_RII"
options(warn=-1)

### Read json data: gagues + events

In [27]:
river_gauges <- fromJSON(paste0(dsn,"\\json\\riv_gauges_parsed.json"), flatten=TRUE)
tidal_gauges <- fromJSON(paste0(dsn,"\\json\\tidal_gauges_parsed.json"), flatten=TRUE)
events <- fromJSON(paste0(dsn,"\\data\\events.json"), flatten=TRUE)$ListEvents

### Clean up events data 

In [ ]:
# Remove wrapping text (i.e., \Data())
events$StartLST <- gsub("[^0-9.-]", "", events$StartLST)
events$EndLST <- gsub("[^0-9.-]", "", events$EndLST)
# Remove everything after the dash
events$StartLST <- gsub("\\-","", events$StartLST)
events$EndLST <- gsub("\\-","", events$EndLST)
# Convert epoch time to human + dataRetrieval readable format 
events$StartDate <- as.POSIXct(as.numeric(as.character(events$StartLST))/10000000, origin = "1970-01-01", tz = "GMT")
events$EndDate <- as.POSIXct(as.numeric(as.character(events$EndLST))/10000000, origin = "1970-01-01", tz = "GMT")
# Drop epoch time columns
events <- events[, !(colnames(events) %in% c("StartLST","EndLST"))]

### Get list of river gauges 

In [ ]:
rg_site_no <- river_gauges$features$properties.Gauge_No

### For g in gauges, for e in events, get max Q 
Max daily flow across duration of event at a particular gauge

In [36]:
k = length(rg_site_no) #10
n = nrow(events) #10

x = matrix(data=NA, nrow=n, ncol=k)

for (g in 1:k){ #length(rg_site_no)
  for (e in 1:n){ #nrow(events)
    siteNumber <- rg_site_no[g]
    parameterCd <- "00060"  # Discharge
    startDate <- events[e,]$StartDate
    endDate <- events[e,]$EndDate
    statCd <- c("00003")  # Maximum
    
    results <- readNWISdv(siteNumber, parameterCd, startDate, endDate, statCd=statCd)
    x[e,g] <- invisible(max(results$X_00060_00003))
  }
}

### Write output to CSV

In [ ]:
write.csv(x, "river_gauges.csv")

## Perform Sanity check

In [17]:
row.names(x) <- events[1:10,"FullEventName"]
colnames(x) <- rg_site_no[1:10]

In [19]:
x

,01482500,01412800,01503000,04263000,04249200,04242500,04242640,04243783,01505000,01510000
NJ Cindy 2005,78.6,89.9,2810,1200,-Inf,-Inf,-Inf,-Inf,194.0,115.0
NJ Gordon 2000,-Inf,-Inf,2240,847,-Inf,-Inf,-Inf,-Inf,-Inf,162.0
NJ Hanna 2008,30.0,56.9,753,1030,-Inf,-Inf,-Inf,-Inf,80.8,48.3
NJ Irene 2011,2630.0,1650.0,24400,2040,-Inf,737.0,-Inf,-Inf,686.0,237.0
NJ Rainfall Jul 2016,12.4,39.7,6960,299,10.8,57.2,155,17.0,143.0,89.9
NJ Rainfall Jul 2017,27.8,75.7,2950,2390,157.0,248.0,661,43.8,376.0,177.0
NJ Riverine Apr 2007,577.0,496.0,12200,3620,-Inf,-Inf,-Inf,-Inf,801.0,540.0
NJ Riverine Aug 2011,1370.0,2710.0,850,949,-Inf,65.3,-Inf,-Inf,47.5,26.3
NJ Riverine Jul 2004,31.7,171.0,1110,1240,-Inf,-Inf,-Inf,-Inf,240.0,171.0
NJ Riverine May 2014,280.0,366.0,5300,5440,1970.0,2450.0,-Inf,-Inf,601.0,430.0


In [20]:
readNWISdv(siteNumber, parameterCd, startDate, endDate, statCd=statCd)

agency_cd,site_no,Date,X_00060_00003,X_00060_00003_cd
USGS,01510000,2014-05-01,430,A


In [28]:
head(events,10)

EventID,State,FullEventName,EventTypes,Days,StartDate,EndDate
1,NJ,NJ Cindy 2005,Tropical or Extratropical,8,2005-07-01,2005-07-13
2,NJ,NJ Gordon 2000,Tropical or Extratropical,7,2000-09-12,2000-09-23
3,NJ,NJ Hanna 2008,Tropical or Extratropical,11,2008-08-26,2008-09-10
4,NJ,NJ Irene 2011,Tropical or Extratropical,9,2011-08-19,2011-09-01
5,NJ,NJ Rainfall Jul 2016,Rainfall,4,2016-07-31,2016-08-03
6,NJ,NJ Rainfall Jul 2017,Rainfall,2,2017-07-29,2017-07-30
7,NJ,NJ Riverine Apr 2007,Riverine,1,2007-04-16,2007-04-16
8,NJ,NJ Riverine Aug 2011,Riverine,1,2011-08-14,2011-08-14
9,NJ,NJ Riverine Jul 2004,Riverine,2,2004-07-13,2004-07-14
10,NJ,NJ Riverine May 2014,Riverine,1,2014-05-01,2014-05-01
